In [7]:
from overcooked_ai_py.agents.benchmarking import AgentEvaluator, LayoutGenerator
from overcooked_ai_py.agents.agent import Agent, AgentPair, StayAgent, GreedyAgent
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld
from overcooked_ai_py.planning.planners import MediumLevelActionManager, COUNTERS_MLG_PARAMS, MotionPlanner

In [ ]:
mdp = OvercookedGridworld.from_layout_name("layout_cesar_0", "./overcooked_ai_py/data/layouts/generation_cesar/")
counter_params = COUNTERS_MLG_PARAMS
if mdp.counter_goals:
    counter_params["counter_goals"] = mdp.counter_goals
    counter_params["counter_drop"] = mdp.counter_goals
    counter_params["counter_pickup"] = mdp.counter_goals
print("Layout chargé avec succès:", mdp.layout_name)
print("Taille du terrain:", len(mdp.terrain_mtx), "x", len(mdp.terrain_mtx[0]))

FileNotFoundError: [Errno 2] No such file or directory: './overcooked_ai_py/data/layouts/generation_cesar_static/layout_0.layout'

In [6]:
# Configuration d'évaluation simplifiée
env_params = {"horizon": 100}  # Réduire l'horizon pour une évaluation plus rapide

# Créer l'évaluateur
agent_eval = AgentEvaluator.from_mdp(mdp, env_params, mlam_params=counter_params)

# Créer deux agents Greedy simples
greedyagent1 = GreedyAgent()
greedyagent1.set_mdp(mdp)
greedyagent2 = GreedyAgent()
greedyagent2.set_mdp(mdp)

# Créer la paire d'agents
agent_pair = AgentPair(greedyagent1, greedyagent2)

print("Démarrage de l'évaluation...")
print(f"Layout: {mdp.layout_name}")
print(f"Horizon: {env_params['horizon']} étapes")

# Lancer l'évaluation avec un horizon réduit
try:
    results = agent_eval.evaluate_agent_pair(agent_pair, num_games=1, native_eval=True)
    print("Évaluation terminée avec succès!")
    print(f"Résultats: {results}")
except Exception as e:
    print(f"Erreur pendant l'évaluation: {e}")
    print("Essayez de réduire encore l'horizon ou d'utiliser des agents plus simples")

  0%|          | 0/1 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Analyse détaillée du layout
print("=== ANALYSE DU LAYOUT ===")
print(f"Nom: {mdp.layout_name}")
print(f"Dimensions: {len(mdp.terrain_mtx)} x {len(mdp.terrain_mtx[0])}")
print(f"Positions de départ: {mdp.start_player_positions}")
print(f"Recettes disponibles: {mdp.start_all_orders}")

# Afficher le terrain
print("\n=== TERRAIN ===")
for i, row in enumerate(mdp.terrain_mtx):
    row_str = ""
    for j, cell in enumerate(row):
        # Ajouter les positions des joueurs
        if (j, i) in mdp.start_player_positions:
            player_num = mdp.start_player_positions.index((j, i)) + 1
            row_str += str(player_num)
        else:
            row_str += cell
    print(row_str)

# Compter les features
pot_count = sum(row.count('P') for row in mdp.terrain_mtx)
onion_count = sum(row.count('O') for row in mdp.terrain_mtx)
tomato_count = sum(row.count('T') for row in mdp.terrain_mtx)
dish_count = sum(row.count('D') for row in mdp.terrain_mtx)
serve_count = sum(row.count('S') for row in mdp.terrain_mtx)

print(f"\n=== FEATURES ===")
print(f"Casseroles (P): {pot_count}")
print(f"Distributeurs d'oignons (O): {onion_count}")
print(f"Distributeurs de tomates (T): {tomato_count}")
print(f"Distributeurs d'assiettes (D): {dish_count}")
print(f"Points de service (S): {serve_count}")

# Vérifier la connectivité basique
empty_spaces = sum(row.count(' ') for row in mdp.terrain_mtx)
total_spaces = len(mdp.terrain_mtx) * len(mdp.terrain_mtx[0])
print(f"\nEspaces vides: {empty_spaces}/{total_spaces} ({empty_spaces/total_spaces*100:.1f}%)")

In [ ]:
# Test simple avec des agents StayAgent (ne bougent pas)
print("=== TEST SIMPLE AVEC AGENTS STATIQUES ===")

# Créer des agents qui ne bougent pas (pour test rapide)
stay_agent1 = StayAgent()
stay_agent2 = StayAgent()
simple_pair = AgentPair(stay_agent1, stay_agent2)

# Test ultra-rapide
simple_env_params = {"horizon": 10}
simple_eval = AgentEvaluator.from_mdp(mdp, simple_env_params, mlam_params=counter_params)

try:
    print("Test avec agents statiques (horizon=10)...")
    simple_results = simple_eval.evaluate_agent_pair(simple_pair, num_games=1, native_eval=True)
    print("✅ Test réussi! Le layout fonctionne correctement.")
    print(f"Score obtenu: {simple_results}")
except Exception as e:
    print(f"❌ Erreur dans le test simple: {e}")
    
print("\nMainte on peut essayer avec des agents plus intelligents...")